In [1]:
import numpy as np
import pandas as pd
import random
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from stable_baselines3.common.vec_env import DummyVecEnv

from env import StockLearningEnv
import config

In [2]:
df = pd.read_csv('./data_file/train.csv')

In [3]:
e_train_gym = StockLearningEnv(df = df, **config.ENV_PARAMS)

加载数据缓存
数据缓存成功!


In [4]:
from stable_baselines3 import DDPG
from stable_baselines3 import A2C
from stable_baselines3 import PPO
from stable_baselines3 import TD3
from stable_baselines3 import SAC

import pandas as pd
import numpy as np
import config

In [5]:
episode = 50000
model = SAC(policy='MlpPolicy', env=e_train_gym, **config.DDPG_PARAMS)
model.learn(total_timesteps= episode)
model.save(os.path.join('train_file', "{}.model".format('SAC' + str(episode))))

EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD|GAINLOSS_PCT|RETREAT_PROPORTION
   0| 499|update         |￥80,184        |￥2,015,551     |101.55513%|101.55513%|0.00%     
   0| 999|update         |￥74,818        |￥2,266,390     |82.60382% |126.63903%|-44.04%   
   0|1473|Last Date      |￥74,818        |￥2,931,466     |165.53440%|193.14664%|-27.61%   
   1| 499|update         |￥62,195        |￥913,177       |-25.60429%|-8.68229% |-16.92%   
   1| 999|update         |￥62,195        |￥980,319       |-14.92637%|-1.96814% |-12.96%   
   1|1499|update         |￥62,195        |￥1,544,905     |8.85620%  |54.49048% |-45.63%   
   1|1999|update         |￥62,195        |￥2,489,598     |136.56957%|148.95978%|-12.39%   
   1|2178|Last Date      |￥62,195        |￥2,189,043     |95.93743% |118.90427%|-22.97%   
   2| 499|update         |￥52,165        |￥1,795,781     |79.57808% |79.57808% |0.00%     
   2| 999|update         |￥52,165        |￥2,118,749     |77.50583% |11

In [6]:
trade_df = pd.read_csv('./data_file/trade.csv')
e_trade_gym = StockLearningEnv(df = trade_df, **config.ENV_TRADE_PARAMS)

加载数据缓存
数据缓存成功!


In [8]:
episode = 50000
model1 = A2C(policy='MlpPolicy', env=e_trade_gym, **config.A2C_PARAMS)
model1.load(os.path.join('train_file', "{}.model".format('A2C' + str(episode))))

model2 = DDPG(policy='MlpPolicy', env=e_trade_gym, **config.DDPG_PARAMS)
model2.load(os.path.join('train_file', "{}.model".format('DDPG' + str(episode))))

model3 = PPO(policy='MlpPolicy', env=e_trade_gym, **config.PPO_PARAMS)
model3.load(os.path.join('train_file', "{}.model".format('PPO' + str(episode))))

model4 = SAC(policy='MlpPolicy', env=e_trade_gym, **config.SAC_PARAMS)
model4.load(os.path.join('train_file', "{}.model".format('SAC' + str(episode))))

model5 = TD3(policy='MlpPolicy', env=e_trade_gym, **config.TD3_PARAMS)
model5.load(os.path.join('train_file', "{}.model".format('TD3' + str(episode))))

In [22]:
def ensemble_pred(models, obs):
    actions = []
    for model in models:
        action, _states = model.predict(obs)
        actions.append(action)
    return np.mean(actions, axis=0)

In [25]:
from tqdm import tqdm

test_env, test_obs = e_trade_gym.get_sb_env()

models = [model1, model2, model3, model4, model5]
test_env.reset()

len_environment = len(e_trade_gym.df.index.unique())
for i in range(len_environment):
    action = ensemble_pred(models, test_obs)
    test_obs, _, dones, _ = test_env.step(action)
    if i == len_environment -2:
        df_account = test_env.env_method(method_name="save_asset_memory")[0]
        df_action = test_env.env_method(method_name="save_action_memory")[0]
        print("回测完成!")

EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD|GAINLOSS_PCT|RETREAT_PROPORTION
回测完成!
   1| 486|Last Date      |￥2,822         |￥1,947,435     |89.71180% |94.74352% |-5.03%    


In [28]:
model_name = 'Ensemble'
df_action.to_csv('./backtest/' + model_name + 'action.csv')

In [29]:
df_account.to_csv('./backtest/' + model_name + 'account.csv')

In [2]:
import numpy as np
A = np.random(10,3)

TypeError: 'module' object is not callable